In [1]:
from statsmodels.stats.multitest import multipletests
from scipy.stats import chi2_contingency
from cellrank.estimators import GPCCA
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.collections as collections 
import scanpy.external as sce
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scrublet as scr
import cellrank as cr
import seaborn as sns
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np
import collections
import matplotlib
#import pyscenic
#import networkx
import anndata
#import network 
#import fsspec
#import igrap
import scvi
import desc
import umap
import sys
import h5py
import os
import re


#%load_ext rpy2.ipython
%matplotlib inline 


sc.settings.verbosity = 3     


# Setting the seed 
np.random.seed(42)


# Setting the figure parameters
sc.set_figure_params(figsize=(2.2, 2.2), format = 'svg', dpi_save = 300)


Global seed set to 0
/Users/brendamelano/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/brendamelano/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [3]:
# List of tuples containing the file paths, cell lines, and drugs
data_info = [
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS052/OS052_ATR_modeling_df.csv', 'OS052', 'ATR'),
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS052/OS052_PF_modeling_df.csv', 'OS052', 'PF'),
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS384/OS384_CIS_modeling_df.csv', 'OS384', 'CIS'),
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS384/OS384_PF_modeling_df.csv', 'OS384', 'PF'),
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS384/OS384_ATR_modeling_df.csv', 'OS384', 'ATR'),
    ('/Users/brendamelano/Desktop/Reprogramming_Osteosarcoma/Lineage_Tracing/scRNAseq_LT_analysis/OS742/OS742_PF_modeling_df.csv', 'OS742', 'PF')
]

# Dictionary to store DataFrames with meaningful keys
dfs = {}

for file_path, cell_line, drug in data_info:
    df = pd.read_csv(file_path)
    df['drug'] = drug
    key = f"{cell_line}_{drug}"
    dfs[key] = df

def process_dataframe(df, name):
    # Ensure 'depleted_10X' is integer
    df['depleted_10X'] = df['depleted_10X'].astype(int)

    # One-hot encode 'State' column
    state_dummies = pd.get_dummies(df['State'])

    # Drop the original 'State' column and add the new one-hot encoded columns
    df = pd.concat([df.drop('State', axis=1), state_dummies], axis=1)

    
    # Apply one-hot encoding to the 'cell_line' column (if necessary)
    df = pd.get_dummies(df, columns=['cell_line'], drop_first=True)

    # Shuffle the DataFrame
    df = df.sample(frac=1).reset_index(drop=True)

    # Create binary columns for State B
    df['State_B'] = (df['State B'] == 1).astype(int)
    df['Not_State_B'] = (df['State B'] != 1).astype(int)

    # Define the predictor and response variables
    X = df[['State_B']]
    X = sm.add_constant(X)  # Add a constant term
    y = df['depleted_10X']

    # Fit the logistic regression model
    logit_model = sm.Logit(y, X)
    try:
        result = logit_model.fit(disp=False)
        print(f"\nLogistic Regression Results for {name}:")
        print(result.summary())
    except Exception as e:
        print(f"An error occurred while processing {name}: {e}")

# Apply the function to each DataFrame
for name, df in dfs.items():
    process_dataframe(df, name)



Logistic Regression Results for OS052_ATR:
                           Logit Regression Results                           
Dep. Variable:           depleted_10X   No. Observations:                 5365
Model:                          Logit   Df Residuals:                     5363
Method:                           MLE   Df Model:                            1
Date:                Tue, 19 Nov 2024   Pseudo R-squ.:                 0.01022
Time:                        17:04:36   Log-Likelihood:                -3582.1
converged:                       True   LL-Null:                       -3619.1
Covariance Type:            nonrobust   LLR p-value:                 8.028e-18
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1617      0.038     -4.236      0.000      -0.236      -0.087
State_B       -0.4811      0.056     -8.560      0.000      -0.591      -0.371

Logisti

In [5]:
# Dictionary to store results for each drug
results_by_drug = {}

for file_path, cell_line, drug in data_info:
    df = pd.read_csv(file_path)
    df['drug'] = drug
    # Ensure 'depleted_10X' is integer
    df['depleted_10X'] = df['depleted_10X'].astype(int)

    # One-hot encode 'State' column
    state_dummies = pd.get_dummies(df['State'])
    states = state_dummies.columns.tolist()

    # Add the new one-hot encoded columns to df
    df = pd.concat([df.drop('State', axis=1), state_dummies], axis=1)

    # Initialize a DataFrame to store odds ratios and p-values for this cell line
    or_pvalues = pd.DataFrame(index=[cell_line], columns=pd.MultiIndex.from_product([states, ['Odds Ratio', 'P-value']]))
    
    for state in states:
        # Create binary predictor for the state
        X = df[[state]]
        X = sm.add_constant(X)  # Add a constant term
        y = df['depleted_10X']

        # Fit the logistic regression model
        logit_model = sm.Logit(y, X)
        try:
            result = logit_model.fit(disp=False)
            odds_ratio = np.exp(result.params[state])
            p_value = result.pvalues[state]
            or_pvalues[(state, 'Odds Ratio')] = odds_ratio
            or_pvalues[(state, 'P-value')] = p_value
        except Exception as e:
            print(f"An error occurred while processing {cell_line} {drug} {state}: {e}")
            or_pvalues[(state, 'Odds Ratio')] = np.nan
            or_pvalues[(state, 'P-value')] = np.nan

    # Store the results
    key = drug
    if key not in results_by_drug:
        results_by_drug[key] = or_pvalues
    else:
        results_by_drug[key] = pd.concat([results_by_drug[key], or_pvalues])

In [8]:
# Now, for each drug, create a heatmap
for drug, df in results_by_drug.items():
    # Reshape the DataFrame for plotting
    odds_ratios = df.xs('Odds Ratio', level=1, axis=1).astype(float)
    p_values = df.xs('P-value', level=1, axis=1).astype(float)
    
    # Apply significance stars
    significance_levels = p_values.applymap(lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else '')
    
    # Combine odds ratios and significance levels for annotation
    annotations = odds_ratios.round(2).astype(str) + '\n' + significance_levels

    # Create a mask for missing values
    mask = odds_ratios.isnull()

    plt.figure(figsize=(4, 3))
    ax = sns.heatmap(odds_ratios, annot=annotations, fmt='', cmap='coolwarm', center=1, mask=mask, cbar_kws={'label': 'Odds Ratio'})
    plt.title(f'Odds Ratios Heatmap for {drug}', fontsize=14)
    plt.xlabel('State', fontsize=12)
    plt.ylabel('Cell Line', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    # Save the figure
    plt.savefig(f"/Users/brendamelano/Desktop/{drug}_odds_ratio_heatmap.png")
    plt.close()

In [10]:
# Now, for each drug, create a heatmap
for drug, df in results_by_drug.items():
    # Rename 'ATR' to 'ATR-i' and 'PF' to 'CDK 4/6-i'
    display_drug = drug
    if drug == 'ATR':
        display_drug = 'ATR-i'
    elif drug == 'PF':
        display_drug = 'CDK 4/6-i'
    
    # Reshape the DataFrame for plotting
    odds_ratios = df.xs('Odds Ratio', level=1, axis=1).astype(float)
    p_values = df.xs('P-value', level=1, axis=1).astype(float)
    
    # Apply significance stars
    significance_levels = p_values.applymap(lambda p: '***' if p < 0.001 else '**' if p < 0.01 else '*' if p < 0.05 else '')
    
    # Combine odds ratios and significance levels for annotation
    annotations = odds_ratios.round(2).astype(str) + '\n' + significance_levels

    # Create a mask for missing values
    mask = odds_ratios.isnull()

    plt.figure(figsize=(4, 3))
    ax = sns.heatmap(
        odds_ratios, 
        annot=annotations, 
        fmt='', 
        cmap='coolwarm', 
        center=1, 
        mask=mask, 
        cbar_kws={'label': 'Odds Ratio'}
    )
    plt.title(f'Odds Ratios Heatmap for {display_drug}', fontsize=14)
    plt.xlabel('State', fontsize=12)
    plt.ylabel('Cell Line', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    # Save the figure
    plt.savefig(f"/Users/brendamelano/Desktop/{drug}_odds_ratio_heatmap.png")
    plt.close()
